In [ ]:
# Grupppo 1

In [ ]:
import pandas as pd
import numpy as np
import operator

In [58]:
# Importa i dati dal file excel
#Modifica la precisione nella visualizzazione delle cifre
pd.options.display.precision = 10

#Legge il csv e lo stampa
df_DatiSensori = pd.read_excel('Dati-gruppo1.xlsx')

In [62]:
#Mostra tutte le colonne
# pd.set_option('display.max_rows', None)

# dataset iniziale
# df_DatiSensori

# dataset con i duplicati eliminati
df_DatiSensori = df_DatiSensori.drop_duplicates()

In [ ]:
# Query 1

In [ ]:
# query effettuata con un unico comando
# df_Benzene = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False)[['postazione', 'Data', 'C6H6_ug/m3']].head(100)

# creazione di un dataFrame ordinato in modo discendente per i valori di benzene
df_C6H6 = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di benzene
query_C6H6 = df_C6H6[['postazione', 'Data', 'C6H6_ug/m3']]
query_C6H6.head(100)

In [ ]:
# Query 2

In [ ]:
# sensore H2S

# query effettuata con un unico comando
# df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False)[['postazione', 'Data', 'H2S_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2S
df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2S = df_H2S[['postazione', 'Data', 'H2S_ug/m3', 'H2SJ_ug/m3']].head(100)
query_H2S

In [ ]:
# sensore H2SJ

# query effettuata con un unico comando
# df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False)[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2SJ
df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2SJ = df_H2SJ[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
query_H2SJ

In [ ]:
# Query 3

In [ ]:
# sensore VOC

# query effettuata con un unico comando
# df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True)[['postazione', 'Data', 'VOC_ppm']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore VOC
df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore VOC
query_VOC = df_VOC[['postazione', 'Data', 'VOC_ppm']].head(100)
query_VOC

In [ ]:
# sensore PIDVOC

# query effettuata con un unico comando
# df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True)[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore PIDVOC
df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore PIDVOC
query_PIDVOC = df_PIDVOC[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
query_PIDVOC

In [ ]:
# Query 4

In [ ]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_C6H6 = df_DatiSensori[['Data','C6H6_ug/m3']].copy()
df_C6H6['Data'] = pd.to_datetime(df_C6H6['Data'])
df_C6H6 = df_C6H6[df_C6H6['C6H6_ug/m3'].notna()]
indexName = df_C6H6[(df_C6H6['C6H6_ug/m3']==0)].index
df_C6H6.drop(indexName, inplace = True)
df_C6H6 = df_C6H6[['Data', 'C6H6_ug/m3']].resample("H", label = 'right', on = 'Data').mean()

In [ ]:
# Stampa le 50 ore con i livelli di benzene più alti
df_C6H6Max = df_C6H6.sort_values('C6H6_ug/m3', ascending = False).head(50)
df_C6H6Max

In [ ]:
# Stampa le 50 ore con i livelli di benzene più bassi
df_C6H6Min = df_C6H6.sort_values('C6H6_ug/m3', ascending = True).head(50)
df_C6H6Min

In [ ]:
#50 ore con il pù basso livello medio di benzene
df_C6H6 = df_DatiSensori[['ora','C6H6_ug/m3']].copy()
df_ore = df_C6H6.groupby('ora').mean().sort_values(['C6H6_ug/m3'], ascending =True).head(50).reset_index()
df_ore[['ora']]

In [ ]:
# Query 5

In [ ]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_AcidoSolf = df_DatiSensori[['Data','H2S_ug/m3','H2SJ_ug/m3' ]].copy()
df_AcidoSolf['Data'] = pd.to_datetime(df_AcidoSolf['Data'])
df_AcidoSolf = df_AcidoSolf[df_AcidoSolf['H2S_ug/m3'].notna()]
df_AcidoSolf = df_AcidoSolf[df_AcidoSolf['H2SJ_ug/m3'].notna()]
indexName = df_AcidoSolf[(df_AcidoSolf['H2S_ug/m3']==0)].index
indexName = df_AcidoSolf[(df_AcidoSolf['H2SJ_ug/m3']==0)].index
df_AcidoSolf.drop(indexName, inplace = True)
df_AcidoSolf = df_AcidoSolf[['Data', 'H2S_ug/m3', 'H2SJ_ug/m3']].resample("H", label = 'right', on = 'Data').mean()

In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2S più alti
df_H2SMax = df_AcidoSolf[df_AcidoSolf.columns[:-1]].sort_values('H2S_ug/m3', ascending = False).head(50)
df_H2SMax

In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2S più basso
df_H2SMin = df_AcidoSolf[df_AcidoSolf.columns[:-1]].sort_values('H2S_ug/m3', ascending = True).head(50)
df_H2SMin

In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2SJ più alti
df_H2SJMax = df_AcidoSolf['H2SJ_ug/m3']
df_H2SJMax= df_H2SJMax.to_frame().sort_values('H2SJ_ug/m3', ascending = False).head(50)
df_H2SJMax

In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2SJ più bassi
df_H2SJMin = df_AcidoSolf['H2SJ_ug/m3']
df_H2SJMin= df_H2SJMin.to_frame().sort_values('H2SJ_ug/m3', ascending = True).head(50)
df_H2SJMin

In [ ]:
# Query 6

In [ ]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_VOC = df_DatiSensori[['Data','VOC_ppm','PIDVOC_ppb' ]].copy()
df_VOC['Data'] = pd.to_datetime(df_VOC['Data'])
df_VOC = df_VOC[df_VOC['VOC_ppm'].notna()]
df_VOC = df_VOC[df_VOC['PIDVOC_ppb'].notna()]
indexName = df_VOC[(df_VOC['VOC_ppm']==0)].index
indexName = df_VOC[(df_VOC['PIDVOC_ppb']==0)].index
df_VOC.drop(indexName, inplace = True)
df_VOC = df_VOC[['Data', 'VOC_ppm', 'PIDVOC_ppb']].resample("H", label = 'right', on = 'Data').mean()

In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore VOC più alti
df_VOCMax = df_VOC[df_VOC.columns[:-1]].sort_values('VOC_ppm', ascending = False).head(50)
df_VOCMax

In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore VOC più bassi
df_VOCMin = df_VOC[df_VOC.columns[:-1]].sort_values('VOC_ppm', ascending = True).head(50)
df_VOCMin

In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore PIDVOC più alti
df_PIDVOCMax = df_VOC['PIDVOC_ppb']
df_PIDVOCMax= df_PIDVOCMax.to_frame().sort_values('PIDVOC_ppb', ascending = False).head(50)
df_PIDVOCMax

In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore PIDVOC più bassi
df_PIDVOCMin = df_VOC['PIDVOC_ppb']
df_PIDVOCMin= df_PIDVOCMin.to_frame().sort_values('PIDVOC_ppb', ascending = True).head(50)
df_PIDVOCMin

In [ ]:
# Query 7

In [ ]:
#Tre giornate con maggior numero fallimenti (manca il calcolo dei 24 zeri consecutivi)
df_Fail = df_DatiSensori[['Data','TRS_ppb']].copy()
df_Fail['Data'] = pd.to_datetime(df_Fail['Data'].dt.strftime('%d/%m/%Y'))
df_Fail['Giorno'] = df_Fail['Data']
df_Fail = df_Fail[df_Fail['TRS_ppb'].isna()]
# df_Fail = df_Fail[(df_Fail['TRS_ppb']!='0')]
df_Fail = df_Fail.groupby('Giorno').count().sort_values('Data', ascending = False)
errors = df_Fail['TRS_ppb'].ne(df_Fail['TRS_ppb'].shift()).cumsum()[df_Fail['TRS_ppb'].eq(0.0)].value_counts().ge(20).sum
df_Fail.head(3)
errors

In [ ]:
#MarioBross2699 Giornate con fallimenti ND
df_C6H6 = df_DatiSensori[['Data','TRS_ppb','VOC_ppm','C6H6_ug/m3','H2S_ug/m3','H2SJ_ug/m3','PIDVOC_ppb']].copy()
df_C6H6['Giorno'] = pd.to_datetime(df_C6H6['Data'].dt.strftime('%d/%m/%Y'))
df_C6H6 = df_C6H6[(df_C6H6['TRS_ppb'].isna()) | (df_C6H6['VOC_ppm'].isna()) | (df_C6H6['C6H6_ug/m3'].isna()) | (df_C6H6['H2S_ug/m3'].isna()) | (df_C6H6['H2SJ_ug/m3'].isna()) | (df_C6H6['PIDVOC_ppb'].isna())]
df_C6H6 = df_C6H6.groupby('Giorno').count()
df_C6H6

In [ ]:
#Serve trovare anche le righe con 144(Praticamente per 12h della stessa giornata non ho registrato) 0 che sono comunque ND.
df_C6H6_0 = df_DatiSensori[['Data','TRS_ppb','VOC_ppm','C6H6_ug/m3','H2S_ug/m3','H2SJ_ug/m3','PIDVOC_ppb']].copy()
df_C6H6_0['Giorno'] = pd.to_datetime(df_C6H6_0['Data'].dt.strftime('%d/%m/%Y'))
df_C6H6_0['TRS_ppb'].astype(float)
df_C6H6_0['VOC_ppm'].astype(float)
df_C6H6_0['C6H6_ug/m3'].astype(float)
df_C6H6_0['H2S_ug/m3'].astype(float)
df_C6H6_0['H2SJ_ug/m3'].astype(float)
df_C6H6_0['PIDVOC_ppb'].astype(float)
df_C6H6_0 = df_C6H6_0[(df_C6H6_0['TRS_ppb'] == 0) | (df_C6H6_0['VOC_ppm'] == 0) | (df_C6H6_0['C6H6_ug/m3']== 0) | (df_C6H6_0['H2S_ug/m3']== 0) | (df_C6H6_0['H2SJ_ug/m3']== 0) | (df_C6H6_0['PIDVOC_ppb']== 0)]
df_C6H6_0 = df_C6H6_0.groupby('Giorno').count()
df_C6H6_0 = df_C6H6_0[(df_C6H6_0.Data >= 144)]
df_C6H6_0

In [ ]:
#Unisco i dati per i fallimenti(Banale union)
df_fail = pd.concat([df_C6H6, df_C6H6_0])

In [ ]:
#3 giorni con il maggior numero di fallimenti
df_fail.sort_values(['Data'], ascending=False).head(3)

In [ ]:
#3 giorni con il minor numero di fallimenti
df_fail.sort_values(['Data'], ascending=True).head(3)

In [ ]:
# Query 8

In [ ]:
# 3 giornate con il minor numero di fallimenti (manca il calcolo dei 24 zeri consecutivi)
df_Fail = df_DatiSensori[['Data','C6H6_ug/m3']].copy()
df_Fail['Data'] = pd.to_datetime(df_Fail['Data'].dt.strftime('%d/%m/%Y'))
df_Fail['Giorno'] = df_Fail['Data']
df_Fail = df_Fail[df_Fail['C6H6_ug/m3'].isna()]
df_Fail = df_Fail[(df_Fail['C6H6_ug/m3']!='0')]
df_Fail = df_Fail.groupby('Giorno').count().sort_values('Data', ascending = True)
df_Fail.head(3)

In [ ]:
# Query 9, 10 e 11

In [ ]:
# creiamo una lista con tutti gli inquinanti
inquinanti = ['TRS_ppb','VOC_ppm','C6H6_ug/m3','H2S_ug/m3','H2SJ_ug/m3','PIDVOC_ppb']
stato_Inquinanti = ['TRS_stato','VOC_stato','C6H6_stato', 'H2S_stato','H2SJ_stato', 'PIDVOC_stato']

In [19]:
# crea un dataframe che contiene la somma dei fallimenti 
# oltre alle righe con 'ND' che indicano il fallimento nel trasmettere i dati, consideriamo fallimento
# anche la presenza consecutiva massiccia di valori 0
fails = []
df_NFails = pd.DataFrame({'Fallimenti':np.arange(6)})
df_NFails = pd.DataFrame({'Sensori':np.arange(6)})
for column in range(len(inquinanti)):
    df_Fail = df_DatiSensori[[inquinanti[column], stato_Inquinanti[column]]].copy()
    errors = df_Fail[inquinanti[column]].ne(df_Fail[inquinanti[column]].shift()).cumsum()[df_Fail[inquinanti[column]].eq(0.0)].value_counts().ge(24).sum()
    num_ND= df_Fail[stato_Inquinanti[column]].value_counts()["ND"]
    fails.append(num_ND + errors)
    column+1


df_NFails['Sensori'] = inquinanti
df_NFails['Fallimenti'] = fails
df_NFails

,Sensori,Fallimenti
0,TRS_ppb,18239
1,VOC_ppm,18588
2,C6H6_ug/m3,17750
3,H2S_ug/m3,17942
4,H2SJ_ug/m3,18411
5,PIDVOC_ppb,17800


In [ ]:
# Query 9

In [63]:
df_NFails['Dati TOT'] = len(df_DatiSensori)
df_NFails['Media'] = df_NFails['Fallimenti'].div(len(df_DatiSensori))
df_NFails

,Sensori,Fallimenti,Dati TOT,Media
0,TRS_ppb,18239,144004,0.1266562040
1,VOC_ppm,18588,144004,0.1290797478
2,C6H6_ug/m3,17750,144004,0.1232604650
3,H2S_ug/m3,17942,144004,0.1245937613
4,H2SJ_ug/m3,18411,144004,0.1278506153
5,PIDVOC_ppb,17800,144004,0.1236076776


In [ ]:
# Query 10

In [56]:
# Prende il massimo dal dataframe
df_NFails['Fallimenti'].max()

18588

In [ ]:
# Query 11

In [57]:
# Prende il minimo dal dataframe
df_NFails['Fallimenti'].min()

17750

In [1]:
#Ho aggiornato la query 7. Da il warning sulla data ma tant'è. Notate che ho contato un fallimento dopo solo 144 0 consecutivi.
#Il criterio sul come contare i fallimenti deve essere unico! Vedete se dovete modifcarlo nella query.